In [50]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Pricing CBC Slide Duplicate\Pricing CBC Replacement Function.ipynb"
%run "..\general_functions\Extracting Data Functions.ipynb"

In [51]:
from pptx import Presentation
import win32com.client as win32
import pandas as pd
import numpy as np
from pathlib import Path
import re
import sys 
win32c = win32.constants
import shutil
import os
from win32com import client
from pptx.util import Pt
from pptx.enum.text import PP_ALIGN
from pptx.chart.data import CategoryChartData,XyChartData,BubbleChartData
import win32com.client
from pptx.dml.color import RGBColor
from pptx.util import Inches,Cm
from pptx.chart.data import ChartData
from pptx.enum.chart import XL_TICK_LABEL_POSITION
from pptx.enum.chart import XL_LABEL_POSITION
from win32com.client import constants as xl
from pptx.enum.chart import XL_CHART_TYPE
import pickle
from pptx.enum.dml import MSO_LINE,MSO_LINE_DASH_STYLE
import time
import itertools
from pptx.enum.chart import XL_AXIS_CROSSES,XL_LEGEND_POSITION
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict 
from io import BytesIO 
import math
from pptx.enum.shapes import MSO_SHAPE
from pptx.oxml.xmlchemy import OxmlElement
from pptx.enum.text import PP_ALIGN, MSO_ANCHOR
from pptx.oxml.ns import qn
import adodbapi
from concurrent.futures import ThreadPoolExecutor, as_completed

### Parameters

In [52]:
file_path = r"C:\Users\SophieZIMMERMANN\Documents\Slide-Automate\Pricing CBC Slide Duplicate\Pricing CBC Datasets\Edgewell US CVS Brand data.xlsx"


market = []
channel=[]
region=['CVS Corp']
marketsub=" ".join(market+channel+region)
client_manuf = ["Edgewell"]
categories = ["Manual Shave Women"]

pricingPlus = "+15%"
pricingMinus = "-10%"

colorList = [
    RGBColor(0, 160, 151),
    RGBColor(126, 202, 196),
    RGBColor(0, 108, 109),
    RGBColor(146, 208, 80),
    RGBColor(0, 176, 80),
    RGBColor(184, 182, 13),
    RGBColor(0, 142, 135),   # New color close to the first and third colors
    RGBColor(131, 218, 212)  # New color close to the second color
]
currency = '$' 
sign = "before"
data_source = "DATA SOURCE: Consumer Test | January 2025"

server = "powerbi://api.powerbi.com/v1.0/myorg/Edgewell"
dataset_name = "Edgewell US Dataset"
entity_hierarchy = [
    ("Region", region),
    ("Channel", channel),
    ("Market", market)
]
hierarchy_levels = [
    ("Category", categories),
]


path=os.path.join(os.getcwd(),"Pricing CBC Datasets")

# Format months for DAX
end_date = "2024-09-01"
past_12_months = pd.date_range(end=end_date, periods=12, freq='ME').strftime('%b-%y').tolist()
p12m_dax = "{" + ", ".join(f'"{date}"' for date in past_12_months) + "}"

BrandelRun= False
BrandSourcingRun=False
ProductSourcingRun=False
PERun=True
RevenueResponseRun=False
SERun=False

### PBI extract for WoB and GM

In [53]:
if PERun: 
    client_manuf_dax = ", ".join(f'"{x}"' for x in client_manuf)
    conn_str = f"Provider=MSOLAP.8;Data Source={server};Initial Catalog={dataset_name};Timeout=600;"


In [54]:
def execute_dax_query(entity_name, area):
    outputdic = {}
    # Construct filter condition dynamically
    columns = ["Product Names WoB %","Gross Margin %"]
    column_exprs = ", ".join(f'"{col}", COALESCE([{col}], 0)' for col in columns)
    dax_query = f"""
    EVALUATE
    CALCULATETABLE(
        ADDCOLUMNS(
            SUMMARIZE(
                ProductNames,
                ProductNames[ProductName]
                ),
            {column_exprs}
        ),
            ProductNames[SourceName]= "Consumerresearchname",
            TREATAS(
                {{"{categories[0]}"}} , 
                Products[Category]
            ),
            TREATAS(
                {{{client_manuf_dax}}},
                Products[Top Companies]
            ),
            TREATAS(
                {p12m_dax},
                Calendar[MonthYear]
            ),
            TREATAS(
                {{"{entity_name}"}},
                Market[{area}]
            )
    )  
    """
 
    try:
        with adodbapi.connect(conn_str) as conn:
            with conn.cursor() as cursor:
                cursor.execute(dax_query)
                columns = [desc[0] for desc in cursor.description]
                data = cursor.fetchall()
 
                df = pd.DataFrame(data, columns=columns)
                df.columns = df.columns.str.replace(r'.*\[|\]', '', regex=True)
                df = df.loc[~(df.select_dtypes(include='number') == 0).all(axis=1)]
                outputdic = df
                print(f"Query executed successfully for {entity_name}.")
    except adodbapi.DatabaseError as db_error:
        print(f"Database error for {entity_name}: {db_error}")
    except Exception as e:
        print(f"Unexpected error for {entity_name}: {e}")
   
    return outputdic
 
# Process data concurrently
def process_dax_queries(entity_hierarchy):
    with ThreadPoolExecutor(max_workers=8) as executor:
       
        dfs_results = {}
        futures = []
        
        for area, entity_list in entity_hierarchy:
            for entity in entity_list:
                future = executor.submit(execute_dax_query, entity, area)
                futures.append((future, entity, area))

        for future, entity, area in futures:
            df = future.result()
            dfs_results[(entity)] = df

        # Save results
        output_path = rf"{path}\Pricing_CBC.pkl"
        pd.to_pickle(dfs_results, output_path)
        print(f"All DataFrames saved to {output_path}.")
 

if PERun:
    process_dax_queries(entity_hierarchy) 
    pbi = pd.read_pickle("Pricing CBC Datasets/Pricing_CBC.pkl")
    for key, df in pbi.items():
        if not df.empty:
            df.rename(columns={'Product Names WoB %':'WOB%', 'Gross Margin %':'GM%'}, inplace=True)
            pbi[key] = df
        else:
            print(f"Warning: DataFrame for {key} is empty.")

Query executed successfully for CVS Corp.
All DataFrames saved to c:\Users\SophieZIMMERMANN\Documents\Slide-Automate\Pricing CBC Slide Duplicate\Pricing CBC Datasets\Pricing_CBC.pkl.


In [55]:
pbi[marketsub]

,ProductName,WOB%,GM%
0,Schick Hydro Silk Ultimate Pubic Blade Refill...,0.0046,0.7811
1,Schick Intuition Fresh Gardenia Blade Refills...,0.0178,0.8492
4,Schick Hydro Silk Sensitive Blade Refills 4 Pack,0.0284,0.8336
5,Schick Hydro Silk Sensitive Razor + 2 Blades,0.0360,0.7502
9,Billie Disposable 3 Blade The Malibu 3 Pack,0.0146,0.5012
10,Schick Hydro Silk Moisture Blade Refills 4 Pack,0.0264,0.8286
11,Schick Intuition Sensitive Blade Refills 6 Pack,0.0879,0.8333
12,Schick Skintimate 4 Blade Vanilla Sugar Dispos...,0.1327,0.5138
13,Schick Intuition Advanced Moisture Razor + 2 B...,0.0152,0.7140
15,Schick Quattro For Women Blade Refills 8 Pack,0.0134,0.7981


### Read Data in Excel

In [56]:
if BrandelRun:
    sheet_name='Brand elasticity'
    brandElasticity_ori=pd.read_excel(file_path,sheet_name)

In [57]:
def normalize_columns(df):
    column_mapping = {
        'product': 'Product',
        'Product': 'Product',
        'SKU': 'Product',
        # Add more mappings as necessary
    }
    
    # Apply the column mapping
    df.rename(columns=column_mapping, inplace=True)
    
    return df


In [58]:
if BrandSourcingRun:
    plusbranding = pd.read_excel(file_path,"Plus Brand Sourcing").rename(columns={'Unnamed: 0': 'Product'})
    Minusbranding = pd.read_excel(file_path,"Minus Brand Sourcing").rename(columns={'Unnamed: 0': 'Product'})
    plusbrandingfair = pd.read_excel(file_path,"Plus Brand Sourcing Fair share").rename(columns={'Unnamed: 0': 'Product'})
    Minusbrandingfair = pd.read_excel(file_path,"Minus Brand Sourcing Fair share").rename(columns={'Unnamed: 0': 'Product'})
    Minusbrandingfair = Minusbrandingfair[[col for col in Minusbrandingfair.columns if 'Unnamed' not in col]]
    plusbranding = normalize_columns(plusbranding)
    Minusbranding = normalize_columns(Minusbranding)
    plusbrandingfair = normalize_columns(plusbrandingfair)
    Minusbrandingfair = normalize_columns(Minusbrandingfair)

In [59]:
if ProductSourcingRun: 
    plus = pd.read_excel(file_path,"Plus Sourcing")
    Minus = pd.read_excel(file_path,"Minus Sourcing")
    plusfair = pd.read_excel(file_path,"Plus Sourcing Fair share")
    Minusfair = pd.read_excel(file_path,"Minus Sourcing Fair share")
    plus = normalize_columns(plus)
    Minus = normalize_columns(Minus)
    plusfair = normalize_columns(plusfair)
    Minusfair = normalize_columns(Minusfair)

In [60]:
if PERun: 
    pe = pd.read_excel(file_path,"PE")
    group_list = pe['Grouping'].unique().tolist()
    pe = normalize_columns(pe)
 

### Data Cleaning

In [61]:
if PERun:
    from openpyxl import load_workbook
    from openpyxl.utils import get_column_letter

    # Load the workbook and select the sheet
    wb = load_workbook(file_path, data_only=True)
    sheet = wb['PE']
    pe["Share P5 - Base Share"] = pd.Series([float(0)] * len(pe))
    pe["PE P5-P6"] = pd.Series([float(0)] * len(pe))
    # Check for yellow cells and adjust shares
    for row in range(1, len(pe)+2):
        for col in range(1, pe.shape[1]+ 2):
            cell = sheet.cell(row=row, column=col)
            if cell.fill.fgColor.rgb == 'FFFFFF00':
                column_name = sheet.cell(1,col).value 
                col_num=int(column_name[1])
                
                for i in range(5,col_num-1,-1):
                    if i == col_num:
                        pe[f"Share P{i} - Base Share"][row-2] =0
                        pe[f"PE P{i}-P{i+1}"][row-2]=0
                    else:
                        pe[f"Share P{i} - Base Share"][row-2] = pe[f"Share P{i-1} - Base Share"][row-2]
                        pe[f"PE P{i}-P{i+1}"][row-2] =  pe[f"PE P{i-1}-P{i}"][row-2]
                

In [62]:
if PERun:
    pbi[marketsub]["ProductName"]=pbi[marketsub]["ProductName"].str.upper()
    pbi[marketsub]["ProductName"] = pbi[marketsub]["ProductName"].apply(lambda x: ' '.join(str(x).split()))
    pe["Product"]=pe["Product"].str.upper()
    pe["Product"]=pe["Product"].apply(lambda x: ' '.join(str(x).split()))
    pepbijoin = pe.join(pbi[marketsub], lsuffix="Product", rsuffix="ProductName")
    pepbijoin=pd.merge(pe,pbi[marketsub][['ProductName','WOB%','GM%']], how='left', left_on='Product', right_on='ProductName')

In [63]:
if RevenueResponseRun: 
    from openpyxl import load_workbook

    # Load the workbook and select the specific sheet
    wb = load_workbook(file_path, data_only=True)
    sheet = wb['Revenue Response']

    # Create an empty color mask matrix
    bgcol = np.empty([sheet.max_row, sheet.max_column], dtype=object, order='C')

    # Cycle through all cells to get colors
    for row in range(sheet.max_row):
        for column in range(sheet.max_column):
            cell = sheet.cell(row + 1, column + 1)
            
            # Check if the cell font color is not red (hex 'FFFF0000'), if it is, set the value to empty
            color_in_hex_index = cell.value if cell.font.color.rgb != 'FFFF0000' else ''
            
            # Assign the value to the bgcol array
            bgcol[row, column] = str(color_in_hex_index)

    # Convert the bgcol array to a pandas DataFrame
    colormask = pd.DataFrame(bgcol)

    # Print the shape of the DataFrame
    #print(colormask.shape)

    # Adjust columns names and content
    # Define ranges for slicing the DataFrame
    if len([]) == 0:
        SheetContentRanges = range(0, colormask.shape[0]), range(0, colormask.shape[1])

    # Slice the DataFrame according to defined ranges
    colormask = colormask.iloc[SheetContentRanges]

    # Reset index and update column names
    colormask.reset_index(drop=True, inplace=True)
    colormask.columns = colormask.iloc[0]  # Set the first row as the header
    colormask = colormask.iloc[1:, :]  # Remove the first row (header) and the first column

    # Drop specific columns
    colormask = colormask.drop(columns=['Select SKUs\n(with "x")', 'Possible steps (d/u)', 'Base case share'])

    # Display the final DataFrame
    #print(colormask)


In [64]:
def cleaning_revenue(colormask):
    """
    Clean revenue data from a DataFrame.
 
    Parameters:
    - colormask: The DataFrame containing revenue data.
 
    Returns:
    - dfList: A list of cleaned DataFrames.
    """
    dfList = []  # Initialize an empty list to store cleaned DataFrames
    colormask = colormask.loc[:, ~colormask.columns.duplicated(keep='last')]  # Remove duplicate columns
    for i in range(1, colormask.shape[0] + 1):
        df = colormask.iloc[i - 1:i]  # Extract a single row DataFrame
        df = df.replace("", np.nan)  # Replace empty cells with NaN
        df.dropna(axis=1, inplace=True)  # Drop columns with NaN values
        dfList.append(df)  # Append cleaned DataFrame to the list
    return dfList  # Return the list of cleaned DataFrames


In [65]:
if RevenueResponseRun: dfList = cleaning_revenue(colormask)

In [66]:
if SERun: 
    sheet_name='SE'
    SE=pd.read_excel(file_path,sheet_name)


In [67]:
def SE_clean(data):
    new_rows = []

    for _, row in data.iterrows():
        # First row: s1 and s2 
        new_rows.append({'Product':row['Product'],
        'Base Size': row['S1 total'], 
        'New Size': row['S2 total'],
        'Size Change':row['size S2 - Base size'],
        'Volume Index': row['Volume Index 2'],
        'Value Index': row['Value Index 2'],
        'Gross Profit':row['Gross Profit 2'],
        'Size Elasticity': row['SE 2'], 
        'SCD PE':row['SCD PE 2'], 
        'PCD PE':row['PCD PE 2']})
        # Second row: s1 and s3 
        new_rows.append({'Product':row['Product'],
        'Base Size': row['S1 total'],
        'New Size': row['S3 total'], 
        'Size Change':row['size S3 - Base size'],
        'Volume Index': row['Volume Index 3'],
        'Value Index': row['Value Index 3'],
        'Gross Profit':row['Gross Profit 3'],
        'Size Elasticity': row['SE 3'], 
        'SCD PE':row['SCD PE 3'], 
        'PCD PE':row['PCD PE 3']})
        
    # Convert the list of new rows back into a DataFrame
    new_df = pd.DataFrame(new_rows)
    new_df = new_df.dropna(thresh=new_df.shape[1] - 7).reset_index(drop=True)
    return new_df


In [68]:
if SERun: SE_cleaned = SE_clean(SE)

## Duplication Stage

In [69]:
index = [0,1 ,2,3,4,5 ]

duplication = [(len(brandElasticity_ori.columns)+3)//6 if BrandelRun else 0,len(plusbranding.keys())-1 if BrandSourcingRun else 0,len(plus.keys())-1 if ProductSourcingRun else 0, len(group_list) if PERun else 0, colormask.shape[0] if RevenueResponseRun else 0, math.ceil(len(SE_cleaned)/20) if SERun else 0]
section_names = ["Brand Elasticity" if BrandelRun else "","Brand Sourcing Analysis" if BrandSourcingRun else 0,"Product Sourcing Analysis" if ProductSourcingRun else 0,"Price Elasticity Curve" if PERun else 0,"Revenue Response Analysis" if RevenueResponseRun else 0,"Size Elasticity" if SERun else 0]

path = os.getcwd() + '//Pricing CBC base Oct 2024.pptx'
new_pre = os.getcwd() + '//slide duplicated.pptx'


In [70]:
slideDuplication(index,duplication,section_names,path,new_pre)

In [71]:
prs = Presentation(new_pre)

### Slide 1

In [72]:
if BrandelRun:
    for sli in range(duplication[0]):

        start=sli*6
        end=start+3
        brandElasticity= brandElasticity_ori[brandElasticity_ori.columns[start:end]]
        brandElasticity=brandElasticity.dropna()
        
        BrandElasticity(prs,brandElasticity,1,categories[0],position=0+sli) 

### Slide 2

In [73]:
if BrandSourcingRun:BrandSourcingAnalysis(prs,plusbranding,Minusbranding,plusbrandingfair,Minusbrandingfair,duplication[1],position=sum(duplication[:1]))

### Slide 3

In [74]:
if ProductSourcingRun:ProductSourcingAnalysis(prs,plus,Minus,plusfair,Minusfair,duplication[2],position=sum(duplication[:2]))

### Slide 4

In [75]:
if PERun:PriceElasticityCurve(prs, pepbijoin, group_list, duplication[3],position=sum(duplication[:3]))

### Slide 5

In [76]:
if RevenueResponseRun: revenue_response(prs,dfList, duplication[4],position=sum(duplication[:4]))

### Slide 6

In [77]:
if SERun: SE_Slide(prs,SE_cleaned,duplication[5],position=sum(duplication[:5]))

### Final

In [78]:
from datetime import date
today = str(date.today())
outputPath=os.getcwd() + "\\Pricing CBC "+marketsub+" output "+today+".pptx"
prs.save(outputPath)
app = win32.Dispatch("PowerPoint.Application")
#presentation = app.Presentations.Open(outputPath)

In [79]:
def get_shape_number_partial(shapes, text):
    """
    Get the index of a shape with specific text.

    Parameters:
    - shapes (list): A list of shapes in a slide.
    - text (str): The text to search for in the shapes.

    Returns:
    - int or None: The index of the shape if found, None otherwise.
    """
    for shape_index in range(len(shapes)):
        if shapes[shape_index].has_text_frame:  # Check if the shape has a text frame
            if text in shapes[shape_index].text:  # Compare the shape's text with the given text
                return shape_index  # Return the index of the shape if text matches
    return None  # Return None if the shape with the given text is not found

In [80]:
def getChartNum(shapes):
    """
    Extractcharts from a collection of shapes in a PowerPoint slide.

    Parameters:
    - shapes: A collection of shapes in a PowerPoint slide.

    Returns:
    - shape_index: shape number in selecton pane
    """
    shape_index = []
    for shape_index in range(len(shapes)):
        if shapes[shape_index].has_chart:
            return shape_index

In [81]:
if RevenueResponseRun:  
    sls = [sl for sl in prs.slides][sum(duplication[:4]):sum(duplication[:5])]
    shapes = prs.slides[sum(duplication[:4])].shapes
    subtitleNumber = get_shape_number_partial(shapes, "Revenue Response Curve by Price Point")
    chartNumber = getChartNum(shapes)
    print(chartNumber)

In [82]:
if RevenueResponseRun:  
    prod_names=[i.shapes[subtitleNumber].text.split(" | ")[1] for i in sls]
    prod_names=list(set(prod_names))
    len(prod_names)
    filter_dps=pd.read_excel(file_path, sheet_name="PE")
    filter_dps=filter_dps.melt(id_vars=["Product"],value_vars=["P1","P2","P3","P4","P5"])
    filter_dps=filter_dps.groupby("Product")["value"].unique().reset_index()
    filter_dps=dict(zip(filter_dps["Product"],filter_dps["value"]))
    len(filter_dps.keys())
    set(prod_names)==set(filter_dps.keys())
    charts=[s.shapes[chartNumber] for s in sls]
    [float(i[0].replace(currency,"")) for i in charts[0].chart.plots[0].categories.flattened_labels]
    dfs=[]
    for num,chart in enumerate(charts):
        temp=pd.DataFrame()
        for i in chart.chart.series:
            temp[i.name]=i.values
        temp["x_axis"]=[float(j[0].replace(currency,"")) for j in chart.chart.plots[0].categories.flattened_labels]
        prod_name=sls[num].shapes[subtitleNumber].text.split(" | ")[1]
        temp=temp[(temp["x_axis"].isin(filter_dps[prod_name]))|(temp["Volume Ix"]==100)]
        dfs.append(temp)
    len(dfs)       

    org_dfs=[]
    for num,chart in enumerate(charts):
        temp=pd.DataFrame()
        for i in chart.chart.series:
            temp[i.name]=i.values
        temp["x_axis"]=[float(j[0].replace(currency,"")) for j in chart.chart.plots[0].categories.flattened_labels]
        prod_name=sls[num].shapes[subtitleNumber].text.split(" | ")[1]
        org_dfs.append(temp)
    len(org_dfs)       

In [83]:
if RevenueResponseRun:  
  final_res=[]
  dfs=org_dfs
  for num_index,df in enumerate(dfs) :
    df["ZeroPoint"]=np.where((df["Volume Ix"]==100),-1,np.nan)
    df["ZeroPoint"]=df["ZeroPoint"].bfill()
    df["ZeroPoint"]=df["ZeroPoint"].fillna(1)
  

    df["Value Ix delta"]=df["Value Ix"].diff() 
    df["Gross Profit Ix delta"]=df["Gross Profit Ix"].diff()

    df["Value Ix delta"]=df["Value Ix delta"]*df["ZeroPoint"]
    df["Gross Profit Ix delta"]=df["Gross Profit Ix delta"]*df["ZeroPoint"]



    df["color"]=np.select([(df["Value Ix delta"]>0)&(df["Gross Profit Ix delta"]>0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]>0)&(df["Gross Profit Ix delta"]<0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]>0)&(df["Gross Profit Ix delta"]==0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]<0)&(df["Gross Profit Ix delta"]>0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]<0)&(df["Gross Profit Ix delta"]<0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]<0)&(df["Gross Profit Ix delta"]==0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]==0)&(df["Gross Profit Ix delta"]>0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]==0)&(df["Gross Profit Ix delta"]<0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]==0)&(df["Gross Profit Ix delta"]==0)&(df["Value Ix"]>100),
                          (df["Value Ix delta"]>0)&(df["Gross Profit Ix delta"]>0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]>0)&(df["Gross Profit Ix delta"]<0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]>0)&(df["Gross Profit Ix delta"]==0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]<0)&(df["Gross Profit Ix delta"]>0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]<0)&(df["Gross Profit Ix delta"]<0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]<0)&(df["Gross Profit Ix delta"]==0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]==0)&(df["Gross Profit Ix delta"]>0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]==0)&(df["Gross Profit Ix delta"]<0)&(df["Value Ix"]<=100),
                          (df["Value Ix delta"]==0)&(df["Gross Profit Ix delta"]==0)&(df["Value Ix"]<=100)],
                          [
  "Green|Profit & Revenue Increase",
  "Yellow|Revenue Increase & Profit Dilution",
  "Yellow|Revenue Increase & Flat Profit",
  "Yellow|Revenue Dilution & Profit Increase",
  "Red|Revenue & Profit Dilution",
  "Red|Revenue Dilution & Flat Profit",
  "Yellow|Flat Revenue & Profit Increase",
  "Red|Flat Revenue & Profit Dilution",
  "Yellow|Flat Revenue & Profit",
  "Yellow|Profit & Revenue Increase",
  "Yellow|Revenue Increase & Profit Dilution",
  "Yellow|Revenue Increase & Flat Profit",
  "Yellow|Revenue Dilution & Profit Increase",
  "Red|Revenue & Profit Dilution",
  "Red|Revenue Dilution & Flat Profit",
  "Yellow|Flat Revenue & Profit Increase",
  "Red|Flat Revenue & Profit Dilution",
  "Yellow|Flat Revenue & Profit",
  ],"")
    

    df=org_dfs[num_index].merge(df,how="left")

    df=df.reset_index(drop=True)
    df=df.iloc[1:,:]
    df["color"]=df["color"].bfill()
    df["grouping_same_color"]=DetermineShapeNumber(df["color"].apply(lambda x : x.split("|")[0]).to_list())
    groups=df.groupby("grouping_same_color")["color"].apply(lambda x : pd.Series(x).mode()[0]).reset_index()
    df=df.drop(columns=["color"]).merge(groups)
    
    df["shape_number"]=DetermineShapeNumber(df["color"].to_list())
    df["color_num"]=df["color"]+"_"+df["shape_number"].astype(str)
    dist=df["color_num"].to_list()
    full_lenght=len(dist)
    dist_list=[[0,0,dist[0] ]]
    current=dist[0]
    for i in range(len(dist)):
      if current == dist[i]:
        dist_list[-1][1]+=1
      else :
        dist_list.append([i,i+1,dist[i]])
        current=dist[i]
    res=[(int(round(i[0]/len(dist),2)*100),int(round(i[1]/len(dist),2)*100),i[2].split("_")[0]) for i in dist_list ]  
    final_res.append(res)
    

In [84]:
if RevenueResponseRun:  
    full_width=Cm(15.7)
    left_start=Cm(2.7)
    for num,slide in enumerate(sls):
        for sh in final_res[num]:
            added_shape=AddRectangle(slide,left_start + (sh[0]/100) * full_width, ((sh[1]-sh[0])/100) * full_width,sh[2] )
        


In [85]:
if RevenueResponseRun:  prs.save(outputPath)